Давайте решим следующую задачу.<br>
Необходимо написать робота, который будет скачивать новости с сайта Лента.Ру и фильтровать их в зависимости от интересов пользователя. От пользователя требуется отмечать интересующие его новости, по которым система будет выделять области его интересов.<br>


Начнем с загрузки новостей. Для этого нам потребуется метод requests.get(url). Библиотека requests предоставляет серьезные возможности для загрузки информации из Интернет. Метод get получает URL стараницы и возвращает ее содержимое. В нашем случае результат будет получаться в формате html. <br>
Загрузим необходимые библиотеки.

In [1]:
import requests # Загрузка новостей с сайта.
from bs4 import BeautifulSoup # Превращалка html в текст.
import re # Регулярные выражения.

Теперь попробуем загрузить страницу новостей.

In [2]:
requests.get("http://lenta.ru/")

<Response [200]>

Метод <i>requests.get()</i> возвращает объект Response, который содержит большое количество различной информации о загруженной (или незагруженной) странице. В краткой форме отображается только результат выполения запроса. В нашем случае это 200, нет ошибки.<br> 
Посмотрим что результат содержит еще.

In [3]:
%%time
resp=requests.get("https://lenta.ru/news/2018/08/24/clon/")
print("cookies:", resp.cookies)
print("time to download:", resp.elapsed)
print("page encoding", resp.encoding)
print("Server response: ", resp.status_code)
print("Is everything ok? ", resp.ok)
print("Page's URL: ", resp.url)

cookies: <RequestsCookieJar[<Cookie is_mobile=0 for .lenta.ru/>, <Cookie lid=vAsAAIveG2BsNXYRAZmIUwB= for .lenta.ru/>, <Cookie lids=483B356CF7A2730B for .lenta.ru/>]>
time to download: 0:00:00.254351
page encoding utf-8
Server response:  200
Is everything ok?  True
Page's URL:  https://lenta.ru/news/2018/08/24/clon/
Wall time: 274 ms


Но самое для нас интересное хранится в поле <i>text</i>, которое содержит собственно текст html-страницы.

In [4]:
resp.text[:1000]

'<!DOCTYPE html><html xmlns:fb="http://www.facebook.com/2008/fbml" xmlns:og="http://ogp.me/ns#"><head><title>В Сибири нашли подходящих для клонирования древних животных: Наука: Наука и техника: Lenta.ru</title><meta content="В Сибири нашли подходящих для клонирования древних животных: Наука: Наука и техника: Lenta.ru" name="title" /><meta content="text/html; charset=utf-8" http-equiv="Content-Type" />\n<script type="text/javascript">window.NREUM||(NREUM={});NREUM.info={"beacon":"bam-cell.nr-data.net","errorBeacon":"bam-cell.nr-data.net","licenseKey":"66a8d51230","applicationID":"1241738","transactionName":"J19cQUoOWA0ERBoQXhRZUUYXElwOFg==","queueTime":0,"applicationTime":139,"agent":""}</script>\n<script type="text/javascript">(window.NREUM||(NREUM={})).loader_config={xpid:"VQUGU1VRGwICUFBVBAk=",licenseKey:"66a8d51230",applicationID:"1241738"};window.NREUM||(NREUM={}),__nr_require=function(t,e,n){function r(n){if(!e[n]){var i=e[n]={exports:{}};t[n][0].call(i.exports,function(e){var i=t

Количество служебной информации в странице явно превышает объем текста новости. У нас есть два пути: либо использовать библиотеку BeautyfulSoup для получения текста статьи, либо получить текст с использованием регулярных выражений.<br>
Опробуем первый путь. Документация на библиотеку BeautyfulSoup находится <a href="https://www.crummy.com/software/BeautifulSoup/bs4/doc/">здесь</a>.

In [5]:
BeautifulSoup(resp.text, "html.parser").get_text()

'В Сибири нашли подходящих для клонирования древних животных: Наука: Наука и техника: Lenta.ru\n\nГлавноеРоссияМирБывший СССРЭкономикаСиловые структурыНаука и техникаКультураСпортИнтернет и СМИЦенности ПутешествияИз жизниДомСтатьиГалереиВидеоСпецпроектыМоторХочешь видеть только хорошие новости? Жми!Лента добра активирована. Это зона смеха, позитива и единорожек.Лента добра деактивирована. Добро пожаловать в реальный мир.Лента добраВсё о коронавирусеНаука и техника\xa0ВсеНаукаЖизньКосмосОружиеИсторияТехникаГаджетыИгрыСофт\xa0АрхивПоследние новости14:20Кремль прокомментировал санкционное давление США на\xa0Россию14:43Названа дата суда над белорусским оппозиционером Бабарико14:40Одна из\xa0самых известных достопримечательностей мира раскололась и\xa0рухнула в\xa0море14:34Тренер «Фиорентины» остался недоволен формой Кокорина14:33Яна Троянова рассталась с\xa0режиссером Сигаревым14:33Российский школьник выкопал снежный тоннель и\xa0оказался в\xa0ловушке14:33Трансгендерам захотели разрешить у

Да, убрать html-теги получилось. Но их содержимое осталось, в том числе и скрипты.<br>
Опробуем другой путь. Весь текст обычно оформляется тегом параграфа - &lt;p&gt;. Выберем весь текст из этих тегов. Заодно выберем и заголовок статьи, оформленный при помощи <h1>

In [6]:
bs=BeautifulSoup(resp.text, "html.parser") 
title=bs.h1.text
text=BeautifulSoup(" ".join([p.text for p in bs.find_all("p")]), "html.parser").get_text()
print(title, "\n-----\n", text)

В Сибири нашли подходящих для клонирования древних животных 
-----
 Российские палеонтологи обнаружили в Якутии тушу жеребенка, возраст которой достигает 30-40 тысяч лет, а также останки мамонта с мягкими тканями. Об этом сообщается в пресс-релизе на Phys.org. Специалисты отмечают хорошее состояние тела лошади, пролежавшей в вечной мерзлоте. Таким образом, находка является потенциально пригодной для клонирования животного. У найденного ископаемого, относящегося к вымершему виду Equus lenensis, сохранились кожа, шерсть, копыта, хвост и внутренние органы. Возраст жеребенка на момент смерти составлял примерно 2-3 месяца. Причиной смерти, вероятно, является попадание в какую-то «ловушку» естественного происхождения, поскольку видимых повреждений на теле не было. У трупа были взяты образцы шерсти и биологических жидкостей для тщательного генетического анализа. По словам исследователей, на данный момент это самые хорошо сохранившиеся из всех останков древних лошадей. В 2015 году в Якутии пал

Получилось хорошо. Но опробуем второй путь.<br>
Теперь попробуем использовать регулярные выражения в два шага. На первом мы вырежем только саму новость с ее оформлением используя для этого регулярные выражения (библиотека re). На втором шаге мы используем библиотеку BeautifulSoup для "выкусывания" тегов html.

In [7]:
findheaders = re.compile("<h1.+?>(.+)</h1>", re.S)
boa = re.compile('<div class="b-text clearfix js-topic__text" itemprop="articleBody">', re.S)
eoa = re.compile('<div class="b-box">\s*?<i>', re.S)
delscript = re.compile("<script.*?>.+?</script>", re.S)

# Получает текст страницы.
art=requests.get("https://lenta.ru/news/2018/08/24/clon/")
# Находим заголовок.
title = findheaders.findall(art.text)[0]
# Выделяем текст новости.
text = eoa.split(boa.split(art.text)[1])
# Иногда новость оканчивается другим набором тегов.
if len(text)==1:
    text = re.split('<div itemprop="author" itemscope=""', text[0])
# Выкусываем скрипты - BeautifulSoup не справляетсяя с ними.
text = "".join(delscript.split(text[0]))
# Выкусываем остальные теги.
print(BeautifulSoup(title+"\n-----\n"+text, "lxml").get_text())


FeatureNotFound: Couldn't find a tree builder with the features you requested: lxml. Do you need to install a parser library?

Обратите внимание на этот фрагмент.<br>
<i>... видимых повреждений на теле не было.У трупа были взяты образцы шерсти...</i><br>
BeautyfulSoup именно "выкусывает" теги, не заменяя их на пробелы. Иногда это можжет приводить к искожению текста из-за "склеивания" слов.

<font color="red" size="6">Закрепим пройденный материал выполнением небольшого задания.</text>

Теперь напишем функцию, которая выгружает все новости за сутки. <br>
Обратим внимание, что для сайта Lenta.ru можно написать адрес в формате lenta.ru/ГГГГ/ММ/ДД/ (год, месяц, день) и получить все новости за этот день. Попробуем получить все адреса с такой страницы.

In [ ]:
BeautifulSoup(requests.get("http://lenta.ru/2018/08/25/").text, "html.parser").find_all("a")

Кажется, это опять немного не то, что нам нужно. Мы получили все ссылки, находящиеся на боковом меню, ссылки на события сегодняшнего дня и другие ненужные нам вещи. <br>
Смотрим в содержимое html-страницы и обращаем внимание, что все интересные нам ссылки оформлены как заголовки третьего уровня - &lt;h3&gt;. Извлечем все такие фрагменты, а потом извлечем собственно адреса, помеченные атрибутом href тега &lt;a&gt;.

In [ ]:
h3s=BeautifulSoup(requests.get("http://lenta.ru/2020/12/25/").text, "html.parser").find_all("h3")
links=["http://lenta.ru"+l.find_all("a")[0]["href"] for l in h3s]
print(links)

Если теперь написать функцию, которая будет перебирать все адреса и получать из них тексты новостей, то мы получим все новости за определенные сутки. Но это мы сделаем чть позже, а пока просто оформим код загрузки статьи в виде функции.

In [ ]:
# Загрузка статьи по URL.
def getOneLentaArticle(url):
    """ getLentaArticle gets the body of an article from Lenta.ru"""
    # Получает текст страницы.
    resp=requests.get(url)
    # Загружаем текст в объект типа BeautifulSoup.
    bs=BeautifulSoup(resp.text, "html.parser") 
    # Получаем заголовок статьи.
    aTitle=bs.h1.text.replace("\xa0", " ")
    # Получаем текст статьи.
    anArticle=BeautifulSoup(" ".join([p.text for p in bs.find_all("p")]), "html5lib").get_text().replace("\xa0", " ")
    return aTitle, anArticle


Разобравшись с базовыми библиотеками, перейдем теперь к обработке собственно текстов. Самостоятельно это можно сделать прочитав одну из двух книг: <a href='https://miem.hse.ru/clschool/the_book'>поновее</a> и <a href='http://clschool.miem.edu.ru/uploads/swfupload/files/011a69a6f0c3a9c6291d6d375f12aa27e349cb67.pdf'>постарше</a> (в старой хорошо разобраны классификация и кластеризация, в новой - тематическое моделирование и рядом лежит видео лекций).<br>
Для обработки текста проводится два этапа анализа: <b>графематический</b> (выделение предложений и слов) и <b>морфологический</b> (определение начальной формы слова, его части речи и грамматических параметров). Этап синтаксического анализа мы разбирать не будем, так как его информация требуется не всегда.<br>
Задачей графематического анализа является разделение текста на составные части - врезки, абзацы, предложения, слова. В таких задачах как машинный перевод, точность данного этапа может существенно влиять на точность получаемых результатов. Например, точка, используемая для сокращений, может быть воспринята как конец предложения, что полность разорвет его семантику.<br>
Но в некоторых задачах (например нашей) используется подход <b>"мешок слов"</b> - текст воспринимается как неупорядоченное множество слов, для которых можно просто посчитать их частотность в тексте. Данный подход проще реализовать, для него не нужно делать выделение составных частей текста, а необходимо только выделить слова.  Именно этот подход мы и будем использовать.<br>

In [ ]:
import pymorphy2 # Морфологический анализатор.
from collections import Counter # Не считать же частоты самим.
import math # Корень квадратный.

Задачей морфологического анализа является определение начальной формы слова, его части речи и грамматических параметров. В некоторых случаях от слова требуется только начальная форма, в других - только начальная форма и часть речи.<br>
Существует два больших подхода к морфологическому анализу: <b>стемминг</b> и <b>поиск по словарю</b>. Для проведения стемминга оставляется справочник всех окончаний для данного языка. Для пришедшего слова проверяется его окончание и по нему делается прогноз начальной формы и части речи.<br>
Например, мы создаем справочник, в котором записываем все окончания прилагательных: <i>-ому, -ему, -ой, -ая, -ий, -ый, ...</i> Теперь все слова, которые имеют такое окончание будут считаться прилагаельными: <i>синий, циклический, красного, больному</i>. Заодно прилагательными будут считаться причастия (<i>делающий, строившему</i>) и местоимения (<i>мой, твой, твоему</i>). Также не понятно что делать со словами, имеющими пустое окончание. Отдельную проблему составляют такие слова, как <i>стекло, больной, вина</i>, которые могут разбираться несколькими вариантами (это явление называется <b>омонимией</b>). Помимо этого, стеммер может просто откусывать окончания, оставляя лишь псевдооснову.<br>
Большинство проблем здесь решается, но точность работы бессловарных стеммеров находится на уровне 80%. Чтобы повысить точность испольуют морфологический анализ со словарем. Разработчики составляют словарь слов, встретившихся в текстах (<a href="http://opencorpora.org/dict.php">здесь</a> можно найти пример такого словаря). Теперь каждое слово будет искаться в словаре и не предсказываться, а выдаваться точно. Для слов, отсутствующих в словаре, может применяться предсказание, пообное работе стеммера.<br>
Посмотрим как работает словарная морфология на примере системы pymorphy2.

In [ ]:
morph=pymorphy2.MorphAnalyzer() # Создает объект морфоанализатора и загружет словарь.
wordform=morph.parse('стекло')  # Проведем анализ слова "стекло"...
print(wordform)                 # ... и посмотрим на результат.

Как видно из вывода, слово "стекло" может быть неодушевленным существительным среднего рода, единственного числа, именительного падежа <i>tag=OpencorporaTag('NOUN,inan,neut sing,nomn')</i>, аналогично, но в винительном падеже (<i>'NOUN,inan,neut sing,accs'</i>), и глаголом <i>'VERB,perf,intr neut,sing,past,indc'</i>. При этом в первой форме оно встречается в 75% случаев (<i>score=0.75</i>), во второй в 18,75% случаев (<i>score=0.1875</i>), а как глагол - лишь в 6,25% (<i>score=0.0625</i>). Самым простым видом борьбы с омонимией является выбор нулевого элемента из списка, возвращенного морфологическим анализом. Такой подход дает около 90% точности при выборе начальной формы и до 80% если мы обращаем внимание на грамматические параметры.<br><br>
Вместо Pymorphy можно использовать PyMystem. Его плюсом является тот факт, что он сам проводит графематический анализ и снимает омонимию. Используя функцию lemmatize можно получить набор начальных форм слов. Используя функцию analyze можно получить полную информацию о словах.

In [ ]:
import pymystem3
mystem=pymystem3.Mystem()

In [ ]:
print(mystem.lemmatize('эти типы стали есть в цеху'))
print(mystem.analyze('эти типы стали есть в цеху'))

Но мы будем использовать pymorphy, так как он немного пошустрее.<br><br>

In [ ]:
art_title, art_text = getOneLentaArticle("https://lenta.ru/news/2018/02/15/greben/")
print(art_title, "\n-----\n", art_text)

Для новостной заметки можно составить ее словарь, а также посчитать частоты всех слов. В итоге мы получим представление текста в виде вектора. В этом векторе координаты будут называться по соответствующим словам, а смещение по данной координате будет показывать частота. <br>
При составлении словаря будем учитывать только значимые слова - существительные, прилагательные и глаголы. Помимо этого предусмотрим возможность учитывать часть речи слова, прибавляя ее у начальной форме.<br>
Для разделения текста на слова используем простейший алгоритм: слово - это последовательность букв русского алфавита среди которых может попадаться дефис. 

In [ ]:
posConv={'ADJF':'_ADJ','NOUN':'_NOUN','VERB':'_VERB'}
meaningfullPoSes=['ADJF', 'NOUN', 'VERB']

def getArticleDictionary(text, needPos=None):
    words=[a[0] for a in re.findall("([А-ЯЁа-яё]+(-[А-ЯЁа-яё]+)*)", text)]
    reswords=[]

    for w in words:
        wordform=morph.parse(w)[0]
        if wordform.tag.POS in meaningfullPoSes:
            if needPos!=None:
                reswords.append(wordform.normal_form+posConv[wordform.tag.POS])
            else:
                reswords.append(wordform.normal_form)
            
    return Counter(reswords)

stat1=getArticleDictionary(art_text, True)
print(stat1)
    


Более правильным методом является использование CountVectorizer из sklearn.feature_extraction.text. При помощи функции <i>fit_transform</i> можно получить разреженное представление матрицы частот слов. Основная проблема состоит в том, что индексы в матрице представляют собой индексы в словаре переданных текстов. Сам словарь хранится в свойстве <i>vocabulary_</i> и умеет возвращать индекс по слову (но не наоборот).

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

counter=CountVectorizer()
# Просим посчитать частоты слов.
res=counter.fit_transform([art_text])
# Разреженное представление счетчика.
print(res[0][0,:10])
# Можно получить индекс по слову, ...
print(counter.vocabulary_.get('фильм'))
# ... но не наоборот.
print(counter.vocabulary_.get(110))


Более того, CountVectorizer просто выделяет подстроки и ничего не знает про морфологию (ее можно правильно прикрутить, но это хлопотное занятие). Зато он умеет выделять n-граммы (n слов идущих подряд (или даже букв)). Помимо этого, можно попросить выдать все подстроки, создав анализатор. И можно сказать как выделять подстроки при помощи регулярного выражения.

In [ ]:
lyrics="""У тебя в кармане
Два мелка и волшебный камень.
Ты волшебный камень
На Восьмое марта подаришь маме.
А с высокой крыши
Все на свете слышно.
Кто-то хитрый и большой
Наблюдает за тобой."""

# При помощи ngram_range=(1,2) говорим, что хотим извлекать слова и пары слов.
# token_pattern показывает регулярное выражение, которому должны соответствовать слова.
counter12=CountVectorizer(ngram_range=(1,2))#, token_pattern=r'[А-Яа-яЁё]+\-[А-Яа-яЁё]+|[А-Яа-яЁё]+')
# Проводим анализ, получаем список найденных n-грамм.
analyze = counter12.build_analyzer()
print(analyze(lyrics))
# Считаем частоты, видим, что слова не приводились к начальной форме.
res=counter12.fit_transform([lyrics])
print(counter12.vocabulary_.get('маме'))
print(counter12.vocabulary_.get('мама'))

Давайте попробуем прикрутить к нему морфологию, но сделаем это немного по-своему. Разобьем строку на слова, проведем их морфологический анализ, возьмем начальные формы и снова склеим в строку, которую отдадим в CountVectorizer. И сделаем тоже самое, только для значимых слов. Как видно из результата, текст перестал быть про "тебя", а стал о чем-то немного другом.

In [ ]:
def getMeaningfullWords(text):
    words=[]
    tokens=re.findall('[А-Яа-яЁё]+\-[А-Яа-яЁё]+|[А-Яа-яЁё]+', text)
    for t in tokens:
        pv=morph.parse(t)
        for p in pv:
            if p.tag.POS in ['ADJF', 'NOUN', 'VERB']:
                words.append(p.normal_form)
                break
    return words

c=[' '.join(getMeaningfullWords(lyrics))]
c2=[' '.join([morph.parse(r)[0].normal_form for r in re.findall('[А-Яа-яЁё]+\-[А-Яа-яЁё]+|[А-Яа-яЁё]+', lyrics)])]

lemmaCounter=CountVectorizer(ngram_range=(1,2), token_pattern=r'[А-Яа-яЁё]+\-[А-Яа-яЁё]+|[А-Яа-яЁё]+')

analyze = lemmaCounter.build_analyzer()
res1=analyze(c[0])
res2=lemmaCounter.fit_transform(c)
print({w:res2[0][0,lemmaCounter.vocabulary_[w]] for w in lemmaCounter.vocabulary_ if res2[0][0,lemmaCounter.vocabulary_[w]]>1})
print("---")
res1=analyze(c2[0])
res2=lemmaCounter.fit_transform(c2)
print({w:res2[0][0,lemmaCounter.vocabulary_[w]] for w in lemmaCounter.vocabulary_ if res2[0][0,lemmaCounter.vocabulary_[w]]>1})


Обратите внимание на слово "исполняющий". В исходном тексте его не было.<br>
Оно появилось так как одним из (не самых вероятных) значений слова "и" является сокращение от "исполняющий". Так как для каждого слова я пытался найти его значение как значимой части речи, союз был отброшен, а вот прилагательное осталось.

Но это лирика, а в новостях получается не так чувствительно. Более того, на одно из первых мест выходит устойчивое словосочетание - "домашний арест".

In [ ]:
c=[' '.join(getMeaningfullWords(art_text))]
c2=[' '.join([morph.parse(r)[0].normal_form for r in re.findall('[А-Яа-яЁё]+\-[А-Яа-яЁё]+|[А-Яа-яЁё]+', art_text)])]

lemmaCounter=CountVectorizer(ngram_range=(1,2), token_pattern=r'[А-Яа-яЁё]+\-[А-Яа-яЁё]+|[А-Яа-яЁё]+')

analyze = lemmaCounter.build_analyzer()
res1=analyze(c[0])
res2=lemmaCounter.fit_transform(c)
print({w:res2[0][0,lemmaCounter.vocabulary_[w]] for w in lemmaCounter.vocabulary_ if res2[0][0,lemmaCounter.vocabulary_[w]]>1})
print("---")
res1=analyze(c2[0])
res2=lemmaCounter.fit_transform(c2)
print({w:res2[0][0,lemmaCounter.vocabulary_[w]] for w in lemmaCounter.vocabulary_ if res2[0][0,lemmaCounter.vocabulary_[w]]>1})


Но нам необходимо искать новости, которые интересны пользователю.<p>
Для определения меры сходства двух статей теперь может использоваться косинусная мера сходства, рассчитываемая по следующей формуле: $cos(a,b)=\frac{\sum{a_i * b_i}}{\sqrt {\sum{a_i^2}*\sum{b_i^2}}}$.<br>
Вообще-то, использовать стандартную функцию рассчета косинусной меры сходства из <a href="http://scikit-learn.org/stable/modules/generated/sklearn.metrics.pairwise.cosine_similarity.html">sklearn</a> было бы быстрее. Но в данной задаче нам бы пришлось сводить все словари в один, чтобы на одних и тех же местах в векторе были частоты одних и тех же слов. Чтобы избежать подобной работы, напишем собственную функцию рассчета косинусного расстояния, работающую с разреженными векторами в виде питоновских словарей.

In [ ]:
def cosineSimilarity(a, b):
    if len(a.keys())==0 or len(b.keys())==0:
        return 0
    sumab=sum([a[na]*b[na] for na in a.keys() if na in b.keys()])
    suma2=sum([a[na]*a[na] for na in a.keys()])
    sumb2=sum([b[nb]*b[nb] for nb in b.keys()])
    return sumab/math.sqrt(suma2*sumb2)


Кстати, вы обратили внимание на красивую формулу в маркдауне выше? Здесь можно использовать html и LaTex для красивого оформления. Например, тег &lt;b&gt;позволяет сделать текст <b>полужирным</b>.

Посчитаем значение косинусной меры для разных статей.

In [ ]:
stat2=getArticleDictionary(getOneLentaArticle("https://lenta.ru/news/2018/02/15/pengilly_domoi/")[1], True)
stat3=getArticleDictionary(getOneLentaArticle("https://lenta.ru/news/2018/02/15/tar_mor/")[1], True)
stat4=getArticleDictionary(getOneLentaArticle("https://lenta.ru/news/2018/02/15/olympmovies/")[1], True)

print(cosineSimilarity(stat1, stat2))
print(cosineSimilarity(stat1, stat3))
print(cosineSimilarity(stat2, stat3))
print(cosineSimilarity(stat2, stat4))
print(cosineSimilarity(stat3, stat4))

Получилось, на самом деле, так себе - статьи очень слабо походят друг на друга. Но может быть потом выйдет лучше.<br>

<font color="red" size="6">А теперь немного поупражняемся.</font><br>
Напишите функцию, которая ищет процент пересечения словарей у двух статей с применением MyStem.

Оформим теперь весь необходимый код как класс, который будет загружать новости с сайта, сохранять их в файл и читать из предварительно сохраненного файла.<br>
Класс - это тип, определенный пользователем (программистом). Класс содержит в себе как данные, так и функции, которые работают с этими данными.<br>
Так как это тип, то можно создавать переменные этого типа. Каждая переменная будет хранить и обрабатывать свой набор данных.

In [ ]:
class getNewsPaper:
        
    # Конструктор - вызывается при создании объекта и инициализирует его.
    def __init__(self):
        self.articles=[]     # Загруженные статьи.
        self.titles=[]       # Заголовки статей.
        self.dictionaries=[] # Словари для каждой из статей.
        # Создаем и загружаем морфологический словарь.
        self.morph=pymorphy2.MorphAnalyzer()

    # Загрузка статьи по URL.
    def getLentaArticle(self, url):
        """ getLentaArticle gets the body of an article from Lenta.ru"""
        # Получает текст страницы.
        resp=requests.get(url)
        # Загружаем текст в объект типа BeautifulSoup.
        bs=BeautifulSoup(resp.text, "html5lib") 
        # Получаем заголовок статьи.
        self.titles.append(bs.h1.text.replace("\xa0", " "))
        # Получаем текст статьи.
        self.articles.append(BeautifulSoup(" ".join([p.text for p in bs.find_all("p")]), "html5lib").get_text().replace("\xa0", " "))

    # Загрузка всех статей за один день.
    def getLentaDay(self, url):
        """ Gets all URLs for a given day and gets all texts. """
        try:
            # Грузим страницу со списком всех статей.
            day = requests.get(url) 
            # Получаем фрагменты с нужными нам адресами статей.
            h3s=BeautifulSoup(day.text, "html5lib").find_all("h3")
            # Получаем все адреса на статьи за день.
            links=["http://lenta.ru"+l.find_all("a")[0]["href"] for l in h3s]
            # Загружаем статьи.
            for l in links:
                self.getLentaArticle(l)
        except:
            pass

    # Загрузка всех статей за несколько дней.
    def getLentaPeriod(self, start, finish):
        curdate=start
        while curdate<=finish:
            print(curdate.strftime('%Y/%m/%d')) # Just in case.
            # Список статей грузится с вот такого адреса.
            self.getLentaDay('https://lenta.ru/news/'+curdate.strftime('%Y/%m/%d'))
            curdate+=datetime.timedelta(days=1)

    # Потроение вектора для статьи.
    posConv={'ADJF':'_ADJ','NOUN':'_NOUN','VERB':'_VERB'}
    def getArticleDictionary(self, text, needPos=None):
        words=[a[0] for a in re.findall("([А-ЯЁа-яё]+(-[А-ЯЁа-яё]+)*)", text)]
        reswords=[]
    
        for w in words:
            wordform=self.morph.parse(w)[0]
            try:
                if wordform.tag.POS in ['ADJF', 'NOUN', 'VERB']:
                    if needPos!=None:
                        reswords.append(wordform.normal_form+self.posConv[wordform.tag.POS])
                    else:
                        reswords.append(wordform.normal_form)
            except:
                pass
            
        stat=Counter(reswords)
#        stat={a: stat[a] for a in stat.keys() if stat[a]>1}
        return stat

    # Посчитаем вектора для всех статей.
    def calcArticleDictionaries(self, needPos=None):
        self.dictionaries=[]
        for a in self.articles:
            self.dictionaries.append(self.getArticleDictionary(a, needPos))
            
    # Сохраняем статьи в файл.
    def saveArticles(self, filename):
        """ Saves all articles to a file with a filename. """
        newsfile=open(filename, "w")
        for art in self.articles:
            newsfile.write('\n=====\n'+art)
        newsfile.close()

    # Читаем статьи из файла.
    def loadArticles(self, filename):
        """ Loads and replaces all articles from a file with a filename. """
        newsfile=open(filename, encoding="utf-8")
        text=newsfile.read()
        self.articles=text.split('\n=====\n')[1:]
        for a in self.articles:
            self.titles.append(a.split('\n-----\n')[0])
        newsfile.close()

    # Для удобства - поиск статьи по ее заголовку.
    def findNewsByTitle(self, title):
        if title in self.titles:
            return self.titles.index(title)
        else:
            return -1

def cosineSimilarity(a, b):
    if len(a.keys())==0 or len(b.keys())==0:
        return 0
    sumab=sum([a[na]*b[na] for na in a.keys() if na in b.keys()])
    suma2=sum([a[na]*a[na] for na in a.keys()])
    sumb2=sum([b[nb]*b[nb] for nb in b.keys()])
    return sumab/math.sqrt(suma2*sumb2)

Загрузим статьи.<br>
<b>!!! Настоятельно рекомендую использовать ячейку с загрузкой статей из файла !!!</b>

In [ ]:
# Загрузка статей за заданный период.
# !!! Это рабоатет довольно долго, пользуйтесь сохраненными данными!!!
lenta=getNewsPaper()
lenta.getLentaPeriod(datetime.date(2018, 2, 1), datetime.date(2018, 2, 14))
lenta.saveArticles("lenta2018.txt")
#lenta.loadArticles("lenta2018.txt")
lenta.calcArticleDictionaries()

In [ ]:
lenta=getNewsPaper()
lenta.loadArticles("lenta2018.txt")
lenta.calcArticleDictionaries()

Из чистого любопытства попробуем найти статью, наиболее похожую на данную.

In [ ]:
# Конечно же, правильнее делать это через np.argmax().
i1 = 0
maxCos, maxpos = -1, -1
for i in range(len(lenta.articles)):
    if i != i1:
        c = cosineSimilarity(lenta.dictionaries[i1], lenta.dictionaries[i])
        if c>maxCos:
            maxCos, maxpos = c, i
print(lenta.articles[i1].split('\n-----\n')[0])
print(lenta.articles[maxpos].split('\n-----\n')[0])
print(maxCos, maxpos)

Сходство между статьями достаточно велико. Есть большие шансы за то, что они об одном и том же.<br><br>
Теперь попробуем решить основную задачу.<br>
Пользователь выбирает несколько статей на интересующую его тематику. Пусть это будут олимпиада и выборы.

In [ ]:
likesport=['Власти США обвинили МОК и ФИФА в коррупции', 'Пробирки WADA для допинг-проб оказались бракованными', 'Пожизненно отстраненных российских спортсменов оправдали', 'В Кремле порадовались за оправданных российских спортсменов', 'Россия вернется на первое место Олимпиады-2014', 'МОК разочаровало оправдание российских олимпийцев', 'Мутко загрустил после оправдания российских спортсменов', 'Оправданный призер Сочи-2014 призвал «добить ситуацию» с МОК', 'Путин предостерег от эйфории после оправдания российских олимпийцев', 'Родченков не смог вразумительно ответить на вопросы суда', 'Оправданный россиянин позлорадствовал над делившими медали Игр-2014 иностранцами', 'В CAS отказались считать оправданных россиян невиновными', 'Адвокат Родченкова заговорил о смерти чистого спорта после оправдания россиян', 'Американская скелетонистка сочла россиян ушедшими от законного наказания']
likeelect=['Социологи подсчитали планирующих проголосовать на выборах-2018', 'Собчак пообещала дать Трампу пару советов', 'На выборы президента России пойдут почти 80 процентов избирателей', 'Песков вспомнил предупреждение и отказался комментировать поездку Собчак в США', 'Собчак съездила на завтрак с Трампом и разочаровалась', 'Грудинин уступил в популярности КПРФ', 'Собчак потребовала признать незаконной регистрацию Путина на выборах', 'У Грудинина обнаружили два не до конца закрытых счета в Швейцарии и Австрии', 'Грудинин раскрыл историю происхождения дома в Испании', 'Путина зарегистрировали кандидатом в президенты', 'В Кремле отреагировали на слухи о голосовании Путина в Севастополе', 'Коммунистов вновь обвинили в незаконной агитации за Грудинина', 'ЦИК выявила обман со стороны Грудинина', 'Грудинин ответил на претензии ЦИК', 'Жириновский захотел сбросить ядерную бомбу на резиденцию Порошенко']

Теперь объединим все выбранные тексты в один и посчитаем ветор для него. Сделаем это два раза для выбранных тематик.

In [ ]:
sporttext=' '.join([lenta.articles[lenta.findNewsByTitle(art)] for art in likesport])
sportdict=lenta.getArticleDictionary(sporttext)
electtext=' '.join([lenta.articles[lenta.findNewsByTitle(art)] for art in likeelect])
electdict=lenta.getArticleDictionary(electtext)
#print(sportdict)
#print(electdict)

А теперь отберем все статьи, косинусная мера которых превышает некоторый порог.

In [ ]:
thrs=0.4
thre=0.5
cosess=[lenta.articles[i].split('\n-----\n')[0] for i in range(len(lenta.dictionaries)) if cosineSimilarity(sportdict, lenta.dictionaries[i])>thrs]
print(cosess)
cosese=[lenta.articles[i].split('\n-----\n')[0] for i in range(len(lenta.dictionaries)) if cosineSimilarity(electdict, lenta.dictionaries[i])>thre]
print(cosese)

Для проверки загрузим новости за какой-то другой день.

In [ ]:
lenta_new=getNewsPaper()
#lenta_new.getLentaPeriod(datetime.date(2018, 2, 15), datetime.date(2018, 2, 15))
#lenta_new.saveArticles("lenta20180215.txt")
lenta_new.loadArticles("lenta20180215.txt")
lenta_new.calcArticleDictionaries()

А теперь проверим какие новости будут находиться.

In [ ]:
thrs_new = 0.3
thre_new = 0.3
cosess_new = [lenta_new.articles[i].split('\n-----\n')[0] for i in range(len(lenta_new.dictionaries)) if cosineSimilarity(sportdict, lenta_new.dictionaries[i])>thrs_new]
print(cosess_new)
cosese_new = [lenta_new.articles[i].split('\n-----\n')[0] for i in range(len(lenta_new.dictionaries)) if cosineSimilarity(electdict, lenta_new.dictionaries[i])>thre_new]
print(cosese_new)

Как видно, метод нуждается в более точном подборе и корректировке параметров.

Вместо того, чтобы выделять самые частотные слова в одной статье, мы можем использовать несколько иную логику. Слово является важным, если оно часто встречается в <b>данной</b> статье. Однако если это слово часто встречается во всех статьях, скорее всего оно не значимо (так ведут себя предлоги, союзы, слова, относящиеся к особенностям авторского или принятого в данной области стиля). То есть наиболее характерными для данного документа будут слова и словосочетания, которые встречаются часто в данном докумемнте, но в малом количестве документов.<br>
Для выделения таких слов будем использовать TfidfVectorizer, работающий по формуле tf*idf, где tf - term frequency, а idf - inverted document frequency. То есть в простейшем случае можно просто разделить частоту термина в документе на количество документов, в которых он встречается.

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np

In [ ]:
c=[' '.join(getMeaningfullWords(n)) for n in lenta.articles]

tfCounter=TfidfVectorizer(ngram_range=(1,2), token_pattern=r'[А-Яа-яЁё]+\-[А-Яа-яЁё]+|[А-Яа-яЁё]+')

analyze = tfCounter.build_analyzer()
res=tfCounter.fit_transform(c)

In [ ]:
id_article=6

res2=analyze(lenta.articles[id_article])
tfs=list(set(res[id_article][0, tfCounter.vocabulary_.get(k)] for k in res2 if k in tfCounter.vocabulary_.keys()))
tfs2=[k for k in tfs if k>np.average(tfs)]
#tfs2=[k for k in tfs if k>np.average(tfs)+np.std(tfs)]
print({w:res[id_article][0, tfCounter.vocabulary_[w]] for w in res2 if w in tfCounter.vocabulary_.keys() and res[id_article][0, tfCounter.vocabulary_[w]] in tfs2})
